In [1]:
import re
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
import time

In [2]:
#initiate Selenium and accept Terms of Use
driver = webdriver.Chrome()

try:
    driver.get("https://qcor.cms.gov/")
    ActionChains(driver)\
        .send_keys(Keys.RETURN)\
        .perform()
except:
        pass

checkbox = driver.find_element(by=By.NAME, value="chkCertiCon")
checkbox.click()

button = driver.find_element(by=By.NAME, value="submit")
button.click()

#move through pages to the results of the RHC Active Provider query
time.sleep(2)

fqhc_link = driver.find_element(by=By.CSS_SELECTOR, value='[href="report_select.jsp?which=11"]')
fqhc_link.click()

time.sleep(2)

apsc_fqhc = driver.find_element(by=By.CSS_SELECTOR, value='[href="FQHC_wizard.jsp?which=11&report=active_nh.jsp"]')
apsc_fqhc.click()

time.sleep(2)

year_box = driver.find_element(by=By.NAME, value="year_type")
select_box = Select(year_box)
select_box.select_by_value("CY")
    
run_report = driver.find_element(by=By.CSS_SELECTOR, value='[onclick="javascript: doSubmit();"]')
run_report.click()

time.sleep(2)
    
atl_link = driver.find_element(by=By.CSS_SELECTOR, value='[name="R04"]')
atl_link.click()

time.sleep(2)

tn_link = driver.find_element(by=By.CSS_SELECTOR, value='[name="STN"]')
tn_link.click()

time.sleep(2)

#create soup
content = driver.page_source
soup = BS(content)
driver.close()

#pull out the provider numbers
drill= soup.find_all("a", attrs={"class" : "drill"})
href = [x.get("href") for x in drill]
href_df = pd.DataFrame(href)
prov_nums = href_df[0].str.extract('(\d{6})')
prov_nums = prov_nums.rename(columns = {0 : 'prov_num'}).dropna()

In [5]:
#initiate Selenium and accept Terms of Use
driver = webdriver.Chrome()

try:
    driver.get("https://qcor.cms.gov/")
    ActionChains(driver)\
        .send_keys(Keys.RETURN)\
        .perform()
except:
        pass

checkbox = driver.find_element(by=By.NAME, value="chkCertiCon")
checkbox.click()

button = driver.find_element(by=By.NAME, value="submit")
button.click()

#Create lists for desired facility information
facility_name = []
cert_num = []
facility_type = []
address = []
city_state = []
phone = []
participation_date = []
ownership_type = []

for ind, row in prov_nums.iterrows():
    #access FQHC provider facility numbers
    prov_id = row['prov_num']
  
    #access page with facility details
    driver.get(f"https://qcor.cms.gov/active_popup.jsp?prvdr_intrnl_num={prov_id}&year_value=2022&jump=1&provider=21&which=11")
    
    #create soup
    content = driver.page_source
    soup = BS(content)
    
    #pull out facility details
    tr = soup.find_all("tr")
    name = tr[1].find("td").text.strip()
    cnum = tr[2].find("td").text.strip()
    ftype = tr[3].find("td").text.strip()
    addr = tr[4].find("td").text.strip()
    cit_st = tr[5].text.strip()
    pho = tr[6].find("td").text.strip()
    pdate = tr[7].find("td").text.strip()
    otype = tr[9].find("td").text.strip()

    #append lists
    facility_name.append(name)
    cert_num.append(cnum)
    facility_type.append(ftype)
    address.append(addr)
    city_state.append(cit_st)
    phone.append(pho)
    participation_date.append(pdate)
    ownership_type.append(otype)

driver.close()

#create dictionary
fqhc_dict = {
    "facility_name" : facility_name,
    "cert_num" : cert_num,
    "facility_type" : facility_type,
    "address" : address,
    "city_state" : city_state,
    "phone" : phone,
    "participation_date" : participation_date,
    "ownership_type" : ownership_type
}

#turn dictionary into a dataframe
fqhc_qcor_df = pd.DataFrame(fqhc_dict)

In [6]:
fqhc_qcor_df

,facility_name,cert_num,facility_type,address,city_state,phone,participation_date,ownership_type
0,AREA HEALTH CENTER,441825,Federally Qualified Health Center,ROUTE 2 BOX 73,"HUNTSVILLE, TN 37756",615 663-2920,06/01/1992,This category is not currently supported
1,BLEDSOE COUNTY PRIMARY CENTER,441866,Federally Qualified Health Center,"3625 US 127, P O BOX 976","PIKEVILLE, TN 37367",615 338-8995,07/25/1994,This category is not currently supported
2,BLUFF CITY MEDICAL CLINIC,441814,Federally Qualified Health Center,229 HIGHWAY 19-E,"BLUFF CITY, TN 37618",615 538-5116,10/01/1991,This category is not currently supported
3,BOGARD PRIMARY CARE CLINIC,441859,Federally Qualified Health Center,2920 COSBY ROAD,"COSBY, TN 37722",615 487-2222,04/01/1992,This category is not currently supported
4,BULLS GAP MEDICAL CLINIC,441812,Federally Qualified Health Center,260 HIGHWAY 11E,"BULLS GAP, TN 37711",615 235-4191,10/01/1991,This category is not currently supported
...,...,...,...,...,...,...,...,...
161,WAVERLY BELMONT MEDICAL CENTER,441821,Federally Qualified Health Center,2507 12TH AVENUE S,"NASHVILLE, TN 37204",615 269-3461,10/01/1991,This category is not currently supported
162,WHITE COUNTY HEALTH DEPT,441847,Federally Qualified Health Center,135 WALKER STREET,"SPARTA, TN 38583",615 836-2201,10/01/1991,This category is not currently supported
163,WILLIAM B ACREE CLINIC,441827,Federally Qualified Health Center,134 NORTH MAIN STREET,"RIDGELY, TN 38080",901 264-5518,02/01/1992,This category is not currently supported
164,WINFIELD MEDICAL CENTER,441869,Federally Qualified Health Center,HIGHWAY 27,"WINFIELD, TN 37892",615 569-4709,05/17/1995,This category is not currently supported
